# Objective 
- In this task, we need to implement Byte Pair Encoding (BPE) from scratch in Python to tokenize Roman Urdu diary entries. The aim is to preprocess the text, which reduces it to a unique set of characters, hence providing the basic vocabulary he used, and then iteratively to pair together the most frequent characters until he has a vocabulary of 1000 tokens. The resulting BPE model should then be applied to a separate set of 14-day diary entries to evaluate the effectiveness of vocabulary size reduction and coverage of Out-of-Vocabulary word. Some of the key challenges include identifying


In [ ]:
import numpy as np
import pandas as pd
import re
import os
from collections import Counter

- This code below sets the dataset directory path and defines a normalization_dict to standardize common spelling variations in Roman Urdu. The normalize_text function processes a given text by splitting it into words and replacing any found in the dictionary with their standardized forms. If a word is not in the dictionary, it remains unchanged. This helps maintain consistency in text preprocessing, making it easier for further analysis, such as tokenization in Byte Pair Encoding (BPE) as shown in the screenshot below.


In [2]:
DATASET_DIR = "C:/Users/user1/Downloads/BPE_dataset/dataset"

In [26]:
normalization_dict = {
    "idhr": "idhar", "udhr": "udhar", "mein": "may", "gya": "gaya",
    "aya": "aaya", "apna": "apni", "kr": "kar", "bht": "bohot",
    "mujy": "mujhe", "tm": "tum", "der": "dair"
}


- This code defines two functions for text preprocessing and dataset loading. The preprocess_text function removes numbering, converts text to lowercase, eliminates punctuation, and applies normalization using the normalize_text function. The load_dataset function reads all .txt files from a specified directory, processes their content using preprocess_text, and combines them into a single text string. This ensures the dataset is cleaned and standardized before further processing.


In [5]:
def normalize_text(text):
    """Normalize spelling variations."""
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return " ".join(normalized_words)

In [18]:
def preprocess_text(text):
    """Preprocess text by removing numbers, converting to lowercase, and normalizing."""
    text = re.sub(r'^\d+\.\s*', '', text.lower())  # Remove numbering and lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = normalize_text(text)  # Normalize spelling
    return text

- The code below defines two functions for handling character-level vocabulary in text processing. The extract_unique_chars function retrieves all unique characters from the dataset and sorts them for consistency. The build_initial_vocab function creates a character-to-ID mapping, assigning unique IDs to each character while reserving ID 0 for an unknown token (<UNK>). This helps in encoding text data for further processing, such as tokenization in a Byte Pair Encoding (BPE) algorithm.

In [34]:
def load_dataset(directory):
    """Load all text files from the dataset directory."""
    all_text = ""
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):  # Ensure only text files are read
            with open(os.path.join(directory, filename), "r", encoding="utf-8", errors="ignore") as file:
                all_text += preprocess_text(file.read()) + " "  # Concatenate all text
    return all_text.strip()

In [35]:
def extract_unique_chars(text):
    """Extract unique characters from the dataset."""
    return sorted(set(text))  # Sort for consistency

def build_initial_vocab(text):
    """Build initial vocabulary with unique characters."""
    unique_chars = extract_unique_chars(text)
    char_to_id = {char: idx+1 for idx, char in enumerate(unique_chars)}  # Assign IDs from 1 onwards
    char_to_id["<UNK>"] = 0  # Assign ID 0 for unknown token
    return char_to_id

In [21]:
corpus = load_dataset(DATASET_DIR)
print(f"Total corpus length: {len(corpus)} characters")

Total corpus length: 82692 characters


s.

## **Initialization (`__init__` method)**
- Sets up the **vocabulary size**.
- Initializes the **final vocabulary**.
- Stores the **merge history** for tracking merges.
- Creates a **token-to-ID mapping** for efficient encoding.

## **Building Initial Vocabulary (`build_initial_vocab` method)**
- Splits words into **character sequences**.
- Counts **word frequencies** to determine token importance.
- Represents each word as a **sequence of individual characters**.
- Forms the **starting vocabulary** for the BPE algorithm.

## **Training BPE (`train_bpe` method)**
- Iteratively merges the **most frequent adjacent character pairs**.
- Continues merging until the vocabulary reaches the **desired size**.
- Maintains an **ordered list of merges** for consistency.
- Updates the **vocabulary** after each merge.
- Constructs a **token-to-ID mapping**, ensuring:
  - **Single characters are added first**.
  - **Merged tokens are added next**.
- Optimizes subword representation for **efficient tokenization**.

## **Pair Frequency Calculation (`get_pair_frequencies` method)**
- Scans the vocabulary to count occurrences of **adjacent character pairs**.
- Identifies the **most frequent** character pairs for merging.

## **Vocabulary Merging (`merge_vocab` method)**
- Merges the **most frequent character pair** into a **new token**.
- Updates all word representations in the vocabulary.
- Iteratively refines the vocabulary to form **larger subword units**.
- Improves **text tokenization efficiency**.

## **Encoding Words (`encode` method)**
- Tokenizes a given word using the **trained BPE model**.
- Splits the word into **individual characters** initially.
- Iteratively applies **learned merges** in the correct order.
- Converts subword tokens into **token IDs**:
  - Uses `<UNK>` token ID for **unseen tokens**.
- Ensures **consistent tokenization** for text processing.

## **Decoding Words (`decode` method)**
- Converts **BPE tokens back into words** by joining subword units.

## **Testing on Unseen Data (`test_bpe_on_diary` method)**
- Evaluates BPE on **unseen diary entries**.
- Counts **total tokens** and **unknown tokens**.
- Computes the **percentage of unknown tokens** to assess model performance.




In [1]:
class BytePairEncoding:
    def __init__(self, vocab_size=1000):
        self.vocab_size = vocab_size
        self.vocab = {}  # Final vocabulary
        self.merges = []  # Ordered list of merges
        self.token_to_id = {}  # Mapping of tokens to IDs
    
    def build_initial_vocab(self, text):
        """Build initial vocabulary with words split into characters."""
        words = text.split()
        word_freqs = Counter(words)
        
        # Initialize vocabulary with character sequences
        vocab = {}
        for word, freq in word_freqs.items():
            # Split word into characters with spaces between them
            chars = ' '.join(list(word))
            vocab[chars] = freq
        
        return vocab
    
    def get_pair_frequencies(self, vocab):
        """Count occurrences of adjacent character pairs."""
        pairs = Counter()
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs
    
    def merge_vocab(self, pair, vocab):
        """Merge most frequent pair into a new token in all words."""
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        new_vocab = {}
        
        for word, freq in vocab.items():
            # Split the word into parts
            parts = word.split()
            
            # Create a new word by merging the pair
            i = 0
            new_parts = []
            while i < len(parts) - 1:
                if parts[i] == pair[0] and parts[i + 1] == pair[1]:
                    new_parts.append(replacement)
                    i += 2
                else:
                    new_parts.append(parts[i])
                    i += 1
            
            # Add the last part if there is one
            if i == len(parts) - 1:
                new_parts.append(parts[-1])
            
            new_word = ' '.join(new_parts)
            new_vocab[new_word] = freq
        
        return new_vocab
    
    def train_bpe(self, text):
        #Train BPE model with ordered merges.
        # Initialize with character-level vocabulary
        vocab = self.build_initial_vocab(text)
        
        # Get current vocabulary size (unique tokens)
        unique_tokens = set()
        for word in vocab.keys():
            unique_tokens.update(word.split())
        
        # Train until we reach desired vocabulary size
        while len(unique_tokens) < self.vocab_size:
            pair_freqs = self.get_pair_frequencies(vocab)
            if not pair_freqs:
                break
                
            # Find most frequent pair
            best_pair = max(pair_freqs, key=pair_freqs.get)
            
            # Add to merges list (preserve order)
            self.merges.append(best_pair)
            
            # Update vocabulary
            vocab = self.merge_vocab(best_pair, vocab)
            
            # Update unique tokens
            unique_tokens = set()
            for word in vocab.keys():
                unique_tokens.update(word.split())
            
            print(f"Vocabulary size: {len(unique_tokens)}, Best pair: {best_pair}")
            
            # Break if we've reached vocabulary size
            if len(unique_tokens) >= self.vocab_size:
                break
        
        # Build final token to ID mapping
        self.token_to_id = {"<UNK>": 0}
        idx = 1
        
        # Add single characters first
        for token in sorted(unique_tokens):
            if len(token) == 1:  # Single character
                self.token_to_id[token] = idx
                idx += 1
        
        # Add merged tokens
        for token in sorted(unique_tokens):
            if len(token) > 1 and token not in self.token_to_id:
                self.token_to_id[token] = idx
                idx += 1
        
        print(f"Final vocabulary size: {len(self.token_to_id)}")
        return self.token_to_id
    
    def encode(self, word):
        """Convert word to BPE tokens using learned merges."""
        if not word:
            return []
            
        # Start with characters separated by spaces
        word = ' '.join(list(word))
        
        # Apply merges in the same order as learned during training
        for pair in self.merges:
            bigram = ' '.join(pair)
            replacement = ''.join(pair)
            
            # Apply non-overlapping merges
            parts = word.split()
            i = 0
            new_parts = []
            while i < len(parts) - 1:
                if parts[i] == pair[0] and parts[i + 1] == pair[1]:
                    new_parts.append(replacement)
                    i += 2
                else:
                    new_parts.append(parts[i])
                    i += 1
            
            # Add the last part if there is one
            if i == len(parts) - 1:
                new_parts.append(parts[-1])
            
            word = ' '.join(new_parts)
        
        # Convert to tokens
        tokens = word.split()
        
        # Convert to token IDs (use UNK for unknown tokens)
        token_ids = []
        for token in tokens:
            if token in self.token_to_id:
                token_ids.append(self.token_to_id[token])
            else:
                token_ids.append(self.token_to_id["<UNK>"])
        
        return tokens, token_ids
    
    def decode(self, tokens):
        """Convert BPE tokens back to word."""
        return "".join(tokens)


In [43]:
def test_bpe_on_diary(directory, bpe_model):
    #Test BPE on unseen diary entries.
    unk_count = 0
    total_tokens = 0
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as file:
                text = preprocess_text(file.read())
                words = text.split()
                
                for word in words:
                    tokens, token_ids = bpe_model.encode(word)
                    total_tokens += len(token_ids)
                    
                    # Count unknown tokens (ID 0)
                    unk_count += token_ids.count(0)
    
    print(f"Total tokens: {total_tokens}, Unknown tokens: {unk_count}")
    print(f"Percentage of unknown tokens: {(unk_count / total_tokens) * 100:.2f}%")

- This code below in the screenshot loads a text dataset from the specified directory and preprocesses it into a single corpus. It then initializes a Byte Pair Encoding (BPE) model with a vocabulary size of 1000 and trains it on the corpus to learn token merges, creating an optimized vocabulary for text compression and representation.


In [45]:
# Load corpus
corpus = load_dataset(DATASET_DIR)
print(f"Total corpus length: {len(corpus)} characters")

# Train BPE model
bpe = BytePairEncoding(vocab_size=1000)
bpe.train_bpe(corpus)



Total corpus length: 82692 characters
Vocabulary size: 39, Best pair: ('h', 'a')
Vocabulary size: 40, Best pair: ('a', 'y')
Vocabulary size: 41, Best pair: ('k', 'a')
Vocabulary size: 42, Best pair: ('h', 'i')
Vocabulary size: 43, Best pair: ('k', 'i')
Vocabulary size: 44, Best pair: ('b', 'a')
Vocabulary size: 45, Best pair: ('u', 'r')
Vocabulary size: 46, Best pair: ('m', 'a')
Vocabulary size: 47, Best pair: ('n', 'a')
Vocabulary size: 48, Best pair: ('h', 'o')
Vocabulary size: 49, Best pair: ('a', 'r')
Vocabulary size: 50, Best pair: ('l', 'a')
Vocabulary size: 51, Best pair: ('k', 'e')
Vocabulary size: 52, Best pair: ('n', 'e')
Vocabulary size: 53, Best pair: ('a', 'ur')
Vocabulary size: 54, Best pair: ('y', 'a')
Vocabulary size: 55, Best pair: ('g', 'ay')
Vocabulary size: 56, Best pair: ('a', 'a')
Vocabulary size: 57, Best pair: ('m', 'e')
Vocabulary size: 58, Best pair: ('i', 'n')
Vocabulary size: 59, Best pair: ('ka', 'r')
Vocabulary size: 60, Best pair: ('s', 'e')
Vocabulary si

{'<UNK>': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '_': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 'é': 38,
 '00': 39,
 '10': 40,
 '100': 41,
 '11': 42,
 '1100': 43,
 '1130': 44,
 '12': 45,
 '1200': 46,
 '15': 47,
 '20': 48,
 '230': 49,
 '30': 50,
 '40': 51,
 '45': 52,
 '50': 53,
 '515': 54,
 '600': 55,
 '630': 56,
 '700': 57,
 '730': 58,
 '830': 59,
 '900': 60,
 'aa': 61,
 'aad': 62,
 'aaj': 63,
 'aakar': 64,
 'aake': 65,
 'aam': 66,
 'aan': 67,
 'aas': 68,
 'aat': 69,
 'aaya': 70,
 'aayi': 71,
 'ab': 72,
 'abhi': 73,
 'ac': 74,
 'acha': 75,
 'achay': 76,
 'achi': 77,
 'ad': 78,
 'ada': 79,
 'adaa': 80,
 'af': 81,
 'ag': 82,
 'agaya': 83,
 'agaye': 84,
 'age': 85,
 'agle': 86,
 'ah': 87,
 'ai': 88,
 'aik': 89,
 'a

- This code tests the trained Byte Pair Encoding (BPE) model by encoding and decoding a sample word ("karunga") to verify its tokenization process. It then evaluates the model on unseen text files from a specified directory, measuring the percentage of unknown tokens to assess how well the BPE vocabulary generalizes to new data.


In [53]:
# Test encoding
test_word = "karunga"
encoded_tokens, encoded_ids = bpe.encode(test_word)
print(f"Encoded '{test_word}': {encoded_tokens} (IDs: {encoded_ids})")

# Test decoding
decoded_word = bpe.decode(encoded_tokens)
print(f"Decoded '{encoded_tokens}': {decoded_word}")

# Test on unseen data
TEST_DIR = "C:/Users/user1/Desktop/NLP_A1"
test_bpe_on_diary(TEST_DIR, bpe)

Encoded 'karunga': ['karun', 'ga'] (IDs: [467, 326])
Decoded '['karun', 'ga']': karunga
Total tokens: 3625, Unknown tokens: 2
Percentage of unknown tokens: 0.06%
